In [2]:
# Import Tensorflow 2.0
#%tensorflow_version 2.x
import tensorflow as tf

#Install Stock API Client (forex aggregates not currently implemented however I addressed the issue and waiting for them to get back to me. Using requests for now)
!pip install --upgrade --force-reinstall polygon-api-client
!pip install requests
!pip install matplotlib

#Import other packages
from polygon import RESTClient
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
import os
import io
import time
import random
import datetime
import functools
import csv
import glob
#from google.colab import drive
from tqdm import tqdm
from IPython import display as ipythondisplay

  Using cached polygon_api_client-1.10.1-py3-none-any.whl (39 kB)
  Using cached websockets-11.0.3-cp39-cp39-macosx_11_0_arm64.whl (121 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: websockets
    Found existing installation: websockets 11.0.3
    Uninstalling websockets-11.0.3:
      Successfully uninstalled websockets-11.0.3
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.12.7
    Uninstalling certifi-2022.12.7:
      Successfully uninstalled certifi-2022.12.7
  Attempting uninstall: polygon-api-client
    Found existing installation: polygon-api-client 1.10.1
    Uninstalling polygon-api-client-1.10.1:
      Successfully uninstalled polygon-api-client-1.10.1


# Data Preprocessing and Clean up

## **1.1 TWO FUNCTIONS: ONE TO ACCESS POLYGON API, ANOTHER TO CONVERT TIME STAMP FOR A DATA POINT.**
From 20010-01 to 2020-12 A total 120 Months.
The function access the Polygon API in 1 hour timeframes in a span of a monnth.
Each Monthly Batch comes in as a JSON file which is converted to a pandas DataFrame then Converted into a CSV file.

In [ ]:
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def print_csv(start, end, year, month):
    # API key
    key = '7FYtfcFojteNAujTh8pPfFoZnHnCl89E'  

    endpoint = 'https://api.polygon.io/v2/aggs/ticker/C:GBPJPY/range/1/hour/{dfrom}/{to}?apiKey={key}&limit=50000'
    resp = requests.get(endpoint.format(dfrom = start, to = end, key = key ))

    my_dictionary = {'t': [' '], 'o': [0.0] , 'h': [0.0], 'l': [0.0], 'c': [0.0], 'v': [0.0], 'vw': [0.0], 'avgOHLC': [0.0]}

    for result in resp.json()['results']:
        dt = ts_to_datetime(result["t"])
        avgOHLC = (result['o'] + result['h'] + result['l'] + result['c']) / 4

        my_dictionary['t'].append(dt)             # Time
        my_dictionary['o'].append(result['o'])    # Open Price
        my_dictionary['h'].append(result['h'])    # High Price
        my_dictionary['l'].append(result['l'])    # Low Price
        my_dictionary['c'].append(result['c'])    # Close Price
        my_dictionary['v'].append(result['v'])    # Volume
        my_dictionary['vw'].append(result['vw'])  # Volume Weighted
        my_dictionary['avgOHLC'].append(avgOHLC)  # Average Price

    # converts dictionary to pandas DataFrame
    df = pd.DataFrame(my_dictionary)            
  
    filename = "/drive/My Drive/CSV2/" + year + "-" + month + ".csv"

    # converts pandas DataFrame into a csv file
    df.to_csv(filename)

## **1.2 SCRIPT FOR AUTOMATING A TIMELINE OF 10 YEARS MONTHLY API CALLS**
Resulting in 120 Monthly CSV Files with each file containing 1 hour data points in a span of a month.

In [ ]:
# specify training data timeline: 10 years
# from 2010 to 2020

import datetime
import calendar

year = 2010
month = 1

for i in range(0, 11):

    for j in range(0, 12):
        _, num_days = calendar.monthrange(year + i, month + j)

        start_day = datetime.date(year + i, month + j, 1)
        end_day = datetime.date(year + i, month + j, num_days)

        print_csv(start_day, end_day, str(year + i), str(month + j))


## **1.3 CONCATENATING ALL 120 MONTHLY CSV FILES INTO ONE CSV FILE.**
The resulting csv file has 68,293 data points from 2010-01 to 2020-12 in 1 hour time frames.

In [ ]:
cwd = os.getcwd()

os.chdir('/drive/My Drive/CSV2/')

extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# combines all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
# export to csv
combined_csv.to_csv( "forexDataGBPJYP.csv", index=False, encoding='utf-8-sig')

## **1.4 UPLOADING CSV FILE**

In [1]:
from google.colab import files
uploaded = files.upload()

ModuleNotFoundError: No module named 'google.colab'